<a href="https://colab.research.google.com/github/IshtiSikder/513-replicating-results/blob/edit/513_final_project_replicating_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate expected cost better !!!!!

In [ ]:
#EDITS
#Install gurobi
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import necessary libraries
import sys
import numpy as np
import pandas as pd
from numpy import linalg as la
import gurobipy as gp
from gurobipy import *

In [ ]:
#Activate gurobi license (only if run on google colab)
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'dcee8c55-6742-4816-bae1-a0742c8f71a9')
e.setParam('WLSSECRET', '70f89e9f-2426-41b9-abe7-88c4b6090a45')
e.setParam('LICENSEID',820825)
e.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 820825
Academic license - for non-commercial use only - registered to isikder@uncc.edu


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=820825>

In [ ]:
#Setup the optimization model
New_Model = gp.Model("New Model",env=e) 

In [ ]:
data = pd.DataFrame({'pre-demand':[100,150,0,0,0,0],'pre-demand prob':[0.5,0.5,0,0,0,0],'demand':[650,750,850,950,1050,1150],'demand prob':[0.17,0.27,0.23,0.17,0.12,0.04]})
data

,pre-demand,pre-demand prob,demand,demand prob
0,100,0.5,650,0.17
1,150,0.5,750,0.27
2,0,0.0,850,0.23
3,0,0.0,950,0.17
4,0,0.0,1050,0.12
5,0,0.0,1150,0.04


In [ ]:
K = range(1,1001)
T = [1,2]

In [ ]:
Q = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
v = New_Model.addVars(K,[0,1,2,3,4],lb=0,vtype=GRB.INTEGER)
u = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
x = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
#q = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
c = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
h = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
s = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)

In [ ]:
'''
for k in K:
  if 1<=k<=6:
    x[k,1]=data['demand'][0] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][0]
  elif 7<=k<=12:
    x[k,1]=data['demand'][1] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][1]
  elif 13<=k<=18:
    x[k,1]=data['demand'][2] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][2]
  elif 19<=k<=24:
    x[k,1]=data['demand'][3] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][3]
  elif 25<=k<=30:
    x[k,1]=data['demand'][4] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][4]
  else:
    x[k,1]=data['demand'][5] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
    q[k]=data['demand prob'][5]
'''

"\nfor k in K:\n  if 1<=k<=6:\n    x[k,1]=data['demand'][0] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    q[k]=data['demand prob'][0]\n  elif 7<=k<=12:\n    x[k,1]=data['demand'][1] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    q[k]=data['demand prob'][1]\n  elif 13<=k<=18:\n    x[k,1]=data['demand'][2] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    q[k]=data['demand prob'][2]\n  elif 19<=k<=24:\n    x[k,1]=data['demand'][3] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n    x[k,2] = np.random.choic

In [ ]:
Q_prime = New_Model.addVars(T,[0],lb=0,vtype=GRB.INTEGER)
#u_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)
#v_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)

In [ ]:
bin = New_Model.addVars(K,[0,1,2],vtype=GRB.BINARY)

In [ ]:
c[1] = 30
c[2] = 4*c[1]
s[1] = 300
s[2] = s[1]

for t in T:
  h[t] = c[t]/2

In [ ]:
for k in K:
  hur_dem = np.random.choice(list(data['demand']), 1, list(data['demand prob']))[0]
  #q[k,1] = data.loc[data.index[data['demand']==hur_dem].tolist()[0],'demand prob']*0.5
  #q[k,2] = 0.5
  x[k,1] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] + hur_dem
  x[k,2] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] 
  #x[k,3] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] 
  #x[k,4] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] 

In [ ]:
New_Model.addConstrs(v[k,0]==0 for k in K)
New_Model.addConstrs(bin[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,2]== bin[k,1]for k in K)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

In [ ]:
New_Model.addConstrs(Q[k,1] == Q_prime[1,0] for k in K)
New_Model.addConstrs(Q[k,2] == Q_prime[2,0] for k in K)

'''
New_Model.addConstrs(Q[k,2] == Q_prime[2,1] for k in K[:6])
New_Model.addConstrs(Q[k,2] == Q_prime[2,2] for k in K[6:12])
New_Model.addConstrs(Q[k,2] == Q_prime[2,3] for k in K[12:18])
New_Model.addConstrs(Q[k,2] == Q_prime[2,4] for k in K[18:24])
New_Model.addConstrs(Q[k,2] == Q_prime[2,5] for k in K[24:30])
New_Model.addConstrs(Q[k,2] == Q_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,6] for k in K[30:36])\n'

In [ ]:
'''
New_Model.addConstrs(u[k,2] == u_prime[2,1] for k in K[:6])
New_Model.addConstrs(u[k,2] == u_prime[2,2] for k in K[6:12])
New_Model.addConstrs(u[k,2] == u_prime[2,3] for k in K[12:18])
New_Model.addConstrs(u[k,2] == u_prime[2,4] for k in K[18:24])
New_Model.addConstrs(u[k,2] == u_prime[2,5] for k in K[24:30])
New_Model.addConstrs(u[k,2] == u_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(u[k,2] == u_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(u[k,2] == u_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(u[k,2] == u_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(u[k,2] == u_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(u[k,2] == u_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(u[k,2] == u_prime[2,6] for k in K[30:36])\n'

In [ ]:
'''
New_Model.addConstrs(v[k,2] == v_prime[2,1] for k in K[:6])
New_Model.addConstrs(v[k,2] == v_prime[2,2] for k in K[6:12])
New_Model.addConstrs(v[k,2] == v_prime[2,3] for k in K[12:18])
New_Model.addConstrs(v[k,2] == v_prime[2,4] for k in K[18:24])
New_Model.addConstrs(v[k,2] == v_prime[2,5] for k in K[24:30])
New_Model.addConstrs(v[k,2] == v_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(v[k,2] == v_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(v[k,2] == v_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(v[k,2] == v_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(v[k,2] == v_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(v[k,2] == v_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(v[k,2] == v_prime[2,6] for k in K[30:36])\n'

In [ ]:
New_Model.addConstrs(Q[k,t]+ bin[k,t-1]*v[k,t-1]+(1-bin[k,t])*u[k,t]-bin[k,t]*v[k,t] == x[k,t] for k in K for t in T)

{(1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (4, 1): <gurobi.QConstr Not Yet Added>,
 (4, 2): <gurobi.QConstr Not Yet Added>,
 (5, 1): <gurobi.QConstr Not Yet Added>,
 (5, 2): <gurobi.QConstr Not Yet Added>,
 (6, 1): <gurobi.QConstr Not Yet Added>,
 (6, 2): <gurobi.QConstr Not Yet Added>,
 (7, 1): <gurobi.QConstr Not Yet Added>,
 (7, 2): <gurobi.QConstr Not Yet Added>,
 (8, 1): <gurobi.QConstr Not Yet Added>,
 (8, 2): <gurobi.QConstr Not Yet Added>,
 (9, 1): <gurobi.QConstr Not Yet Added>,
 (9, 2): <gurobi.QConstr Not Yet Added>,
 (10, 1): <gurobi.QConstr Not Yet Added>,
 (10, 2): <gurobi.QConstr Not Yet Added>,
 (11, 1): <gurobi.QConstr Not Yet Added>,
 (11, 2): <gurobi.QConstr Not Yet Added>,
 (12, 1): <gurobi.QConstr Not Yet Added>,
 (12, 2): <gurobi.QConstr Not Yet Added>,
 (13, 1): 

In [ ]:
#New_Model.addConstrs(Q[k,t]+v[k,t-1]+u[k,t]-v[k,t] == x[k,t] for k in K for t in T)

In [ ]:
obj = 0
penalty = 50

for t in T:
  for k in K:
    obj += c[t]*Q[k,t] + h[t]*v[k,t] + s[t]*u[k,t]*penalty

New_Model.setObjective(obj, GRB.MINIMIZE)
New_Model.optimize()


print('Obj: %g' % New_Model.ObjVal)
print('Expected value:', New_Model.ObjVal/len(K))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to isikder@uncc.edu
Optimize a model with 4000 rows, 14008 columns and 6000 nonzeros
Model fingerprint: 0x49c2c986
Model has 2000 quadratic constraints
Variable types: 0 continuous, 14008 integer (3000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+02, 1e+03]
Presolve removed 4000 rows and 8006 columns
Presolve time: 0.05s
Presolved: 10000 rows, 18002 columns, 29000 nonzeros
Presolved model has 8000 SOS constraint(s)
Variable types: 0 continuous, 18002 integer (6000 binary)
Found heuristic solution: objective 1

In [ ]:
print(1,0,Q_prime[1,0].X)
print(2,0,Q_prime[2,0].X)

1 0 1450.0
2 0 0.0


In [ ]:
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',(bin[k,t-1]).X,'*',v[k,t-1].X,'+',(1-bin[k,t].X),'*',u[k,t].X,'-',(bin[k,t].X),'*',v[k,t].X,'=',x[k,t])

scene: 1 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 150.0 = 1300
scene: 1 time: 2 0.0 + 1.0 * 150.0 + 0.0 * -0.0 - 1.0 * 50.0 = 100
scene: 2 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 650.0 = 800
scene: 2 time: 2 -0.0 + 1.0 * 650.0 + 0.0 * -0.0 - 1.0 * 550.0 = 100
scene: 3 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 300.0 = 1150
scene: 3 time: 2 0.0 + 1.0 * 300.0 + 0.0 * -0.0 - 1.0 * 200.0 = 100
scene: 4 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 350.0 = 1100
scene: 4 time: 2 0.0 + 1.0 * 350.0 + 0.0 * -0.0 - 1.0 * 250.0 = 100
scene: 5 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 600.0 = 850
scene: 5 time: 2 0.0 + 1.0 * 600.0 + 0.0 * -0.0 - 1.0 * 450.0 = 150
scene: 6 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 200.0 = 1250
scene: 6 time: 2 0.0 + 1.0 * 200.0 + 0.0 * -0.0 - 1.0 * 100.0 = 100
scene: 7 time: 1 1450.0 + 0.0 * 0.0 + 0.0 * -0.0 - 1.0 * 200.0 = 1250
scene: 7 time: 2 0.0 + 1.0 * 200.0 + 0.0 * -0.0 - 1.0 * 50.0 = 150
scene: 8 time: 1 1450.0 + 0.0 * 0.0 +

In [ ]:
x[1,1]

1300

In [ ]:
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',v[k,t-1].X,'+',u[k,t].X,'-',v[k,t].X,'=',x[k,t])

scene: 1 time: 1 1450.0 + 0.0 + -0.0 - 150.0 = 1300
scene: 1 time: 2 0.0 + 150.0 + -0.0 - 50.0 = 100
scene: 2 time: 1 1450.0 + 0.0 + -0.0 - 650.0 = 800
scene: 2 time: 2 -0.0 + 650.0 + -0.0 - 550.0 = 100
scene: 3 time: 1 1450.0 + 0.0 + -0.0 - 300.0 = 1150
scene: 3 time: 2 0.0 + 300.0 + -0.0 - 200.0 = 100
scene: 4 time: 1 1450.0 + 0.0 + -0.0 - 350.0 = 1100
scene: 4 time: 2 0.0 + 350.0 + -0.0 - 250.0 = 100
scene: 5 time: 1 1450.0 + 0.0 + -0.0 - 600.0 = 850
scene: 5 time: 2 0.0 + 600.0 + -0.0 - 450.0 = 150
scene: 6 time: 1 1450.0 + 0.0 + -0.0 - 200.0 = 1250
scene: 6 time: 2 0.0 + 200.0 + -0.0 - 100.0 = 100
scene: 7 time: 1 1450.0 + 0.0 + -0.0 - 200.0 = 1250
scene: 7 time: 2 0.0 + 200.0 + -0.0 - 50.0 = 150
scene: 8 time: 1 1450.0 + 0.0 + -0.0 - 500.0 = 950
scene: 8 time: 2 0.0 + 500.0 + -0.0 - 350.0 = 150
scene: 9 time: 1 1450.0 + 0.0 + -0.0 - 200.0 = 1250
scene: 9 time: 2 0.0 + 200.0 + -0.0 - 50.0 = 150
scene: 10 time: 1 1450.0 + 0.0 + -0.0 - 450.0 = 1000
scene: 10 time: 2 0.0 + 450.0 + -0

In [ ]:
x[k,t]

150